In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
"""
Survived - 생존유무, target 값. (0 = 사망, 1 = 생존)
Name - 탑승객 성명
Pclass - 티켓 클래스. (1 = 1st, 2 = 2nd, 3 = 3rd)
Sex - 성별
Age - 나이(세)
SibSp - 함께 탑승한 형제자매, 배우자 수 총합
Parch - 함께 탑승한 부모, 자녀 수 총합
Embarked - 탑승 항구
Fare - 탑승 요금
Ticket - 티켓 넘버
Cabin - 객실 넘버
"""

In [ ]:
cd /kaggle/input/titanic

# Data Load & Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns # 데이터 시각화 라이브러리
import matplotlib.pyplot as plt # 그래프 라이브러리
# 그래프 인라인 표시 명령어
%matplotlib inline 

plt.rc('font', family='Malgun Gothic') # matplotlib에서 폰트를 'Malgun Gothic'으로 설정
plt.rc('axes', unicode_minus=False) # matplotlib에서 축의 유니코드 마이너스 기호 사용을 비활성화

import warnings
warnings.filterwarnings(action='ignore') # 경고 메시지를 무시하도록 설정

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('gender_submission.csv')

In [ ]:
train.head()

In [ ]:
test.info()

In [ ]:
print(train.shape)  #(891, 12)
print(test.shape)  #(418, 11)

submission.head()

In [ ]:
# 결측지 확인
print(train.isnull().sum())
print(test.isnull().sum())

# train data에는 Age, Cabin, Embarked 컬럼에 결측치가 존재한다
# test data에는 Age, Fare, Cabin 컬럼에 결측치가 존재한다

In [ ]:
# train 결측치 시각화
import missingno as msno
msno.matrix(train, figsize=(12, 5))

# EDA

In [ ]:
# 생존비율 확인
train["Survived"].value_counts()  # 'Survived' 열의 값이 각각 몇 번씩 등장하는지 세어 출력 (사망자와 생존자 수)

plt.figure(figsize=(8, 5))  # 그래프의 크기를 가로 8인치, 세로 5인치로 설정
labels = ['Survived', 'Dead']  # 파이 차트의 각 섹션에 대한 레이블을 설정

train['Survived'].value_counts().plot.pie(
    explode=[0, 0.08],  # '생존' 섹션을 약간 분리시켜 강조
    shadow=True,  # 파이 차트에 그림자 효과를 추가
    autopct='%1.1f%%',  # 각 섹션의 비율을 소수점 첫째 자리까지 표시
    labels=labels  # 섹션 레이블을 '사망'과 '생존'으로 설정
)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 1행 2열의 서브플롯을 생성, 전체 크기를 가로 12인치, 세로 5인치로 설정

axes[0].set_title("Passengers by Gender \n", size=15)  # 첫 번째 서브플롯의 제목을 '성별 탑승자 수'로 설정, 글자 크기를 15로 지정
sns.countplot(x="Sex", data=train, ax=axes[0])  # 'Sex' 열을 기준으로 탑승자 수를 세어 첫 번째 서브플롯에 countplot을 그림

axes[1].set_title("Survived by Gender \n", size=15)  # 두 번째 서브플롯의 제목을 '성별 생존자 수'로 설정, 글자 크기를 15로 지정
sns.countplot(x="Sex", hue="Survived", data=train, ax=axes[1])  # 'Sex' 열을 기준으로 'Survived'에 따라 그룹화하여 두 번째 서브플롯에 countplot을 그림
axes[1].legend(labels = ['dead', 'survived'])  # 두 번째 서브플롯의 범례를 '사망'과 '생존'으로 설정


In [ ]:
# 성별 생존률
sur = train[['Sex', 'Survived']].groupby('Sex').mean().plot.bar()  # 'Sex' 열을 기준으로 그룹화하여 'Survived' 열의 평균을 계산하고, 이를 막대 그래프로 시각화

# 성별 생존률을 계산하여 출력
train[['Sex', 'Survived']].groupby('Sex').mean()  # 'Sex' 열을 기준으로 그룹화하여 'Survived' 열의 평균을 계산하고, 그 결과를 출력


In [ ]:
# 티켓등급별 생존
train[['Pclass', 'Survived']].groupby('Pclass').mean()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 1행 2열의 서브플롯을 생성, 전체 크기를 가로 12인치, 세로 5인치로 설정

axes[0].set_title("Number of Passengers by Class \n", size=15)  # 첫 번째 서브플롯의 제목을 '클래스별 탑승수'로 설정, 글자 크기를 15로 지정
sns.countplot(data=train, x='Pclass', ax=axes[0])  # 'Pclass' 열을 기준으로 탑승자 수를 세어 첫 번째 서브플롯에 countplot을 그림

axes[1].set_title("Survived/Deceased Count by Class \n", size=15)  # 두 번째 서브플롯의 제목을 '클래스별 생존/사망 수'로 설정, 글자 크기를 15로 지정
sns.countplot(x="Pclass", hue="Survived", data=train, ax=axes[1])  # 'Pclass' 열을 기준으로 'Survived'에 따라 그룹화하여 두 번째 서브플롯에 countplot을 그림
axes[1].legend(labels = ['Deceased', 'Survived'])  # 두 번째 서브플롯의 범례를 '사망'과 '생존'으로 설정


In [ ]:
# class별 생존률
sur = train[['Pclass', 'Survived']].groupby(['Pclass']).mean().plot.bar()
sur.set_title('Survived/Deceased Count by Class', size=15)

In [ ]:
# 나이
train['Age'].describe() 

In [ ]:
plt.figure(figsize=(8, 5))  # 그래프의 크기를 가로 8인치, 세로 5인치로 설정

sns.histplot(train['Age'], bins=25, kde=True)  # 'Age' 열의 분포를 히스토그램과 커널 밀도 추정(KDE)을 사용하여 시각화, 막대(bin)의 개수를 25개로 설정


In [ ]:
plt.figure(figsize=(8, 5))  # 그래프의 크기를 가로 8인치, 세로 5인치로 설정

# 'Survived' 열에 따라 'Age' 분포를 KDE로 시각화, 라벨을 생존 여부에 따라 설정
sns.kdeplot(data=train, x='Age', hue='Survived', bw_adjust=0.5)

# y축 눈금 설정 (0.005 단위)
y_ticks = np.arange(0, 0.04, 0.005)
plt.yticks(y_ticks)

plt.legend(labels=['Survived','Deceased'])  # 범례를 추가하여 생존자와 사망자의 분포를 구분
plt.show()  # 그래프를 출력


In [ ]:
# 동반가족 유무 생존률
print(train[['SibSp', 'Survived']].groupby(['SibSp']).mean())
print(train[['Parch', 'Survived']].groupby(['Parch']).mean())

# 개인적으로 유의미한 데이터인가는 의문

In [ ]:
# 탑승 항구별 생존률
train[['Embarked', 'Survived']].groupby(['Embarked']).mean()

In [ ]:
# 각 Pclass별 Embarked 값의 개수를 세어 데이터프레임으로 변환
pc1 = train[train['Pclass'] == 1]['Embarked'].value_counts()  # 1등급 승객의 탑승 항구 분포
pc2 = train[train['Pclass'] == 2]['Embarked'].value_counts()  # 2등급 승객의 탑승 항구 분포
pc3 = train[train['Pclass'] == 3]['Embarked'].value_counts()  # 3등급 승객의 탑승 항구 분포

# 각 Pclass별 데이터프레임을 결합하여 새로운 데이터프레임 생성
pc = pd.DataFrame([pc1, pc2, pc3])
pc.index = ['1st class', '2nd class', '3rd class']  # 데이터프레임의 인덱스를 객실 등급으로 설정

# 시각화
fig, ax = plt.subplots(figsize=(8, 5))  # 그래프의 크기를 가로 8인치, 세로 5인치로 설정
pc.plot(kind='bar', stacked=True, ax=ax)  # 데이터프레임을 사용하여 막대 그래프 생성, 막대를 쌓아서 표시
plt.show()  # 그래프를 출력


In [ ]:
# 탑승 금액(Fare)
fig, ax = plt.subplots(figsize=(10,6))
 
# 분포 확인
sns.kdeplot(train[train['Survived']==1]['Fare'], ax=ax)
sns.kdeplot(train[train['Survived']==0]['Fare'], ax=ax)
 
# 축 범위
ax.set(xlim=(0, train['Fare'].max()))
ax.legend(['survived', 'dead'])
 
plt.show()

Cabin은 패스

In [ ]:
train.columns
# Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
#        'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='object')

# Feature Engineering

In [ ]:
# 상관계수
numeric_train = train.select_dtypes(include='number')
correlation_matrix = numeric_train.corr()

correlation_matrix

In [ ]:
# 결측치
train.isnull().sum()

In [ ]:
# 데이터 전처리 시작
data_list = [train, test]

In [ ]:
# 성별
train['Sex'].head()

# 범주형(category) 변수 "수치 맵핑 방식" 변환 (String -> int)

# int type의 숫자 0과 1로 매핑
for data in data_list:
    data['Sex'] = data['Sex'].astype('category').cat.codes
    
train['Sex'].head()

In [ ]:
# Age 결측치 평균으로 채우기
sex_mean = train.groupby('Sex')['Age'].mean()
sex_mean

In [ ]:
# Age 결측치 처리
for data in data_list:  # data_list의 각 데이터프레임에 대해 반복
    data.loc[(data['Sex'] == 0) & (data['Age'].isnull()), 'Age'] = sex_mean[0]  # 'Sex'가 0(남성)이고 'Age'가 결측치인 경우, 해당 'Age'를 남성의 평균 나이로 채움
    data.loc[(data['Sex'] == 1) & (data['Age'].isnull()), 'Age'] = sex_mean[1]  # 'Sex'가 1(여성)이고 'Age'가 결측치인 경우, 해당 'Age'를 여성의 평균 나이로 채움    
train.isnull().sum()

# data.loc[행 조건, 열 이름]

In [ ]:
# Age "수치 매핑 변환" 카테고리화

# 'Age' 열을 5개의 구간으로 나누어 'AgeRange' 열에 저장
train['AgeRange'] = pd.cut(train['Age'], 5)

# 'AgeRange' 열과 'Survived' 열을 선택하여, 'AgeRange'별 평균 생존율을 계산
result = train[['AgeRange', 'Survived']].groupby(['AgeRange']).mean()

# 결과 출력
print(result)


In [ ]:
# 16, 32, 48, 64 기준으로 카테고리화
for data in data_list:    
    data.loc[ data['Age'] <= 16, 'Age'] = 0
    data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
    data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
    data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
    data.loc[ data['Age'] > 64, 'Age'] = 4
train.drop('AgeRange', inplace=True, axis=1) # AgeRange열은 삭제
train.head()

In [ ]:
# Name (?)

# 'Name' 열에서 호칭(Title)을 추출하여 'Title' 열에 저장
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False) 
# 정규 표현식으로 공백 문자 뒤에 오는 알파벳 단어와 마침표를 찾는다
# expand = False : 추출된 결과를 데이터프레임이 아닌 시리즈로 반환

# 'Title' 열의 각 값의 빈도수를 계산하여 출력
title_counts = train['Title'].value_counts()

title_counts

In [ ]:
# 개별 호칭들은 Others로

for data in data_list:
    data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    data['Title'] = data['Title'].replace(['Dr', 'Rev', 'Mlle', 'Major', 'Col', 'Countess',
                                        'Capt', 'Ms', 'Sir', 'Lady', 'Mme', 'Don', 'Jonkheer'], 'Other')
    # -> 범주형 -> 수치형
    data['Title_name'] = data['Title'].astype('category').cat.codes

In [ ]:
train[['Title', 'Survived']].groupby(['Title']).mean()

In [ ]:
# 불필요한 데이터 지우기
train.drop(['Name', 'Title'], axis=1, inplace = True)
test.drop(['Name', 'Title'], axis=1, inplace = True)
train.head()

In [ ]:
# 가족수

# 'FamilySize' 열을 생성하여 가족 규모를 계산 ('SibSp' + 'Parch' + 1)
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1  

train.head()


In [ ]:
# 'FamilySize'별 생존율을 계산하여 피벗 테이블 생성
pivot_table = pd.pivot_table(train, index='FamilySize', values='Survived')

# 'FamilySize'별 생존자와 사망자 수를 시각화하는 막대 그래프 생성
sns.countplot(data=train, x='FamilySize', hue='Survived')

# 피벗 테이블 출력 (생존율 확인)
pivot_table


In [ ]:
# 컬럼 제거
drop_list = ['SibSp', 'Parch']
for data in data_list:
    data.drop(drop_list, inplace=True, axis=1)

In [ ]:
# 'Embarked' 열의 결측치를 'S'로 채우기 (결측치 2개라 최빈값 S로)
for data in data_list:  # data_list의 각 데이터프레임에 대해 반복
    data['Embarked'] = data['Embarked'].fillna('S')  # 'Embarked' 열의 결측치를 'S'로 채움

# 'Embarked'별 생존자와 사망자 수를 시각화하는 막대 그래프 생성
sns.countplot(data=train, x='Embarked', hue='Survived')

# 그래프 출력
plt.show()


In [ ]:
# 'Embarked' 열의 값을 숫자로 매핑하기 위한 dictionary
em_mapping = {'S': 0, 'C': 1, 'Q': 2}

# 'Embarked' 열의 값을 숫자로 매핑
for data in data_list:  # data_list의 각 데이터프레임에 대해 반복
    data['Embarked'] = data['Embarked'].map(em_mapping)  # 'Embarked' 열의 값을 em_mapping 사전을 사용하여 숫자로 매핑


In [ ]:
# Cabin
for data in data_list:
    data['Cabin'] = data['Cabin'].fillna('N')
    data['Cabin'] = data['Cabin'].apply(lambda x:x[0])
    data['Cabin'] = data['Cabin'].astype('category').cat.codes
train.head()

# 결측치엔 'N', 각각 맨 앞 알파벳별로 카테고리화 후 수치

In [ ]:
# Fare
# Age와 동일하게 구간 나누어 분류
# 카테고리화
for data in data_list:
    data['Farerange'] = pd.cut(data['Fare'], 4)
train[['Farerange', 'Survived']].groupby(['Farerange']).mean()

for data in data_list:    
    data.loc[data['Fare'] <= 128, 'Fare'] = 0
    data.loc[(data['Fare'] > 128) & (data['Fare'] <= 256), 'Fare'] = 1
    data.loc[(data['Fare'] > 256) & (data['Fare'] <= 384), 'Fare'] = 2
    data.loc[data['Fare'] > 384, 'Fare'] = 3

train.head()

In [ ]:
# 컬럼 지우기
for data in data_list:
    data.drop('Farerange', axis=1, inplace=True)

In [ ]:
train.head()
# 보니깐 PassengerId, Ticket은 필요 x

In [ ]:
drop_list = ['PassengerId', 'Ticket']
for data in data_list:
    data.drop(drop_list, inplace=True, axis=1) # axix=1 : 열(column)을 기준으로 동작
    
for data in data_list:
    data.drop('Cabin', inplace=True, axis=1)

In [ ]:
# test 세트의 결측치 1개도 평균으로 채움
test[test['Fare'].isnull()] = test['Fare'].mean()

In [ ]:
# Feature Engineering 끝
print(train.head())
print(test.head())

In [ ]:
train.corr()

# Modeling

In [ ]:
# 'Survived' 열을 제외한 나머지 열을 입력 데이터로 선택하고, 이를 numpy 배열로 변환
train_input = train.drop('Survived', axis=1).values

# 'Survived' 열을 타겟 데이터로 선택하고, 이를 numpy 배열로 변환
train_target = train['Survived'].values

# 입력 데이터와 타겟 데이터의 shape을 출력
print(train_input.shape, train_target.shape)  # (891, 7) (891,)

# 필요한 라이브러리 임포트
from sklearn.model_selection import KFold  # 교차 검증을 위한 K-Fold 클래스
from sklearn.model_selection import cross_validate  # 교차 검증을 위한 함수
from sklearn.model_selection import cross_val_score  # 교차 검증 점수를 계산하기 위한 함수
from sklearn.model_selection import StratifiedKFold  # 층화된 K-Fold 교차 검증을 위한 클래스
from sklearn.neighbors import KNeighborsClassifier  # K-최근접 이웃 분류기
from sklearn.tree import DecisionTreeClassifier  # 의사결정 트리 분류기
from sklearn.ensemble import RandomForestClassifier  # 랜덤 포레스트 분류기
from sklearn.ensemble import GradientBoostingClassifier  # 그라디언트 부스팅 분류기
from sklearn.experimental import enable_hist_gradient_boosting  # 히스토그램 그라디언트 부스팅 분류기 사용을 위한 모듈
from sklearn.ensemble import HistGradientBoostingClassifier  # 히스토그램 그라디언트 부스팅 분류기
from sklearn.naive_bayes import GaussianNB  # 가우시안 나이브 베이즈 분류기
from sklearn.svm import SVC  # 서포트 벡터 머신 분류기


In [ ]:
# KNN
# K-최근접 이웃 분류기 모델 생성
model = KNeighborsClassifier()

# 교차 검증을 수행하고, 훈련 점수와 테스트 점수를 반환
score = cross_validate(model, train_input, train_target,
                       return_train_score=True, n_jobs=-1,
                       cv=StratifiedKFold())

# n_jobs=-1 : 가능한 모든 CPU 코어를 사용하여 병렬 처리를 수행
# cv=StratifiedKFold() : 층화된 K-폴드 교차 검증을 사용

# 훈련 점수와 테스트 점수의 평균을 계산하여 출력
print(np.mean(score['train_score']), np.mean(score['test_score']))


In [ ]:
# Random Forest
model = RandomForestClassifier()
score = cross_validate(model, train_input, train_target,
                      return_train_score=True, n_jobs=-1,
                      cv = StratifiedKFold())
print(np.mean(score['train_score']), np.mean(score['test_score']))

In [ ]:
# GradientBoosting
model = GradientBoostingClassifier()
score = cross_validate(model, train_input, train_target,
                      return_train_score=True, n_jobs=-1,
                      cv = StratifiedKFold())
print(np.mean(score['train_score']), np.mean(score['test_score']))

In [ ]:
# HistGradientBoosting
model = HistGradientBoostingClassifier()
score = cross_validate(model, train_input, train_target,
                      return_train_score=True, n_jobs=-1,
                      cv = StratifiedKFold())
print(np.mean(score['train_score']), np.mean(score['test_score']))

In [ ]:
# Naive Bayes
model = GaussianNB()
score = cross_validate(model, train_input, train_target,
                      return_train_score=True, n_jobs=-1,
                      cv = StratifiedKFold())
print(np.mean(score['train_score']), np.mean(score['test_score']))

In [ ]:
# Support Vector Machine(SVM)
model = SVC()
score = cross_validate(model, train_input, train_target,
                      return_train_score=True, n_jobs=-1,
                      cv = StratifiedKFold())
print(np.mean(score['train_score']), np.mean(score['test_score']))

Train과 Val의 격차가 가장 적은 SVM 채택

In [ ]:
# SVM 분류기 모델 생성 및 훈련
model = SVC()  # K-최근접 이웃 분류기 인스턴스 생성
model.fit(train_input, train_target)  # train_input(X)과 train_target(Y)을 사용하여 모델 훈련

# 테스트 데이터 입력값 설정
test_input = test.values  # 테스트 데이터프레임을 numpy 배열로 변환

# 테스트 데이터에 대한 예측 수행
pred = model.predict(test_input)  # 테스트 데이터에 대한 예측값 생성

# 제출 파일 생성
submission['Survived'] = pred  # 예측값을 'Survived' 열에 저장
submission  # 제출 파일 확인


최종 제출 csv 생성

In [ ]:
cd /kaggle/working # working dir이 쓰기 권한 허용

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
sub = pd.read_csv("submission.csv")

In [ ]:
sub.head()

In [ ]:
ls